In [1]:
import sys
import random
import pathlib
import pandas as pd

project_root = pathlib.Path.cwd().resolve()
while not (project_root / "backend").is_dir():
    if project_root.parent == project_root:
        raise RuntimeError("Could not find project root containing 'backend/'")
    project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from backend.utils import fetch_metrics
from backend import constants, data_retrieval, data_push


random_num = random.randint(0, 50)

country_data_df = pd.read_excel(r"/home/linux/Projects/AI-Country-Risk-Dashboard/backend/data/country_data.xlsx")
countries: list = country_data_df["Country_Name"].values.tolist()

# Testing Program for Random Country
selected_country = countries[random_num]
country_iso_code = country_data_df[country_data_df["Country_Name"] == selected_country]["iso2Code"].values.tolist()[0]

# Fetch All Data Points For A Random Country
country_data = fetch_metrics.build_country_panel(country_iso_code, constants.INDICATORS)
print(f"Country: {selected_country} | Country ISO Code: {country_iso_code}")
country_data

Country: United States | Country ISO Code: US


,INFLATION,UNEMPLOYMENT,FDI_PCT_GDP,POL_STABILITY,RULE_OF_LAW,CONTROL_CORRUPTION,GINI_INDEX,GDP_PC_GROWTH,INT_PAYM_PCT_REV
1960,1.457976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,1.070724,NaN,NaN,NaN,NaN,NaN,NaN,0.879102,NaN
1962,1.198773,NaN,NaN,NaN,NaN,NaN,NaN,4.509854,NaN
1963,1.239669,NaN,NaN,NaN,NaN,NaN,36.7,2.866168,NaN
1964,1.278912,NaN,NaN,NaN,NaN,NaN,37.4,4.303810,NaN
...,...,...,...,...,...,...,...,...,...
2020,1.233584,8.055,0.641872,-0.027900,1.336494,1.037832,40.0,-2.561807,10.903066
2021,4.697859,5.349,2.009212,-0.014425,1.389157,1.019278,39.7,5.888341,11.395363
2022,8.002800,3.650,1.572591,0.008701,1.369202,1.104145,41.7,1.923862,13.405689
2023,4.116338,3.638,1.258207,0.029425,1.327678,1.123433,41.8,2.035600,17.983682


In [2]:
country_context = data_retrieval.prepare_llm_payload_pretty(
    country_iso=country_iso_code,
    indicators=constants.INDICATORS,
    since=2020,
    lookback=5
    )
country_context

{'country': 'US',
 'latest_year': 2024,
 'indicators': {'Inflation (% y/y)': {'latest': 2.95,
   'Δ1y': -0.283,
   'Δ5y': None,
   'series': {2020: 1.23, 2021: 4.7, 2022: 8.0, 2023: 4.12, 2024: 2.95}},
  'Unemployment (% labour force)': {'latest': 4.11,
   'Δ1y': 0.129,
   'Δ5y': None,
   'series': {2020: 8.06, 2021: 5.35, 2022: 3.65, 2023: 3.64, 2024: 4.11}},
  'FDI inflow (% GDP)': {'latest': 1.33,
   'Δ1y': 0.057,
   'Δ5y': None,
   'series': {2020: 0.64, 2021: 2.01, 2022: 1.57, 2023: 1.26, 2024: 1.33}},
  'Political stability (z-score)': {'latest': None,
   'Δ1y': None,
   'Δ5y': None,
   'series': {2020: -0.03, 2021: -0.01, 2022: 0.01, 2023: 0.03}},
  'Rule of law (z-score)': {'latest': None,
   'Δ1y': None,
   'Δ5y': None,
   'series': {2020: 1.34, 2021: 1.39, 2022: 1.37, 2023: 1.33}},
  'Control of corruption (z-score)': {'latest': None,
   'Δ1y': None,
   'Δ5y': None,
   'series': {2020: 1.04, 2021: 1.02, 2022: 1.1, 2023: 1.12}},
  'Income inequality (Gini)': {'latest': None,
 

In [3]:
"""
Example pipeline:
1. Pull structured macro data  ➜  prompt_points
2. Fetch fresh Google-News headlines
3. Ask the LLM for a risk score & summary
"""

import json
from datetime import timezone, datetime

from backend import constants
from backend import data_retrieval
from backend.ai import langchain_llm
from backend.utils import fetch_links

# ---------------------------------------------------------------------------
# 1. Build the macro-data payload (World Bank panel) -------------------------
# ---------------------------------------------------------------------------

COUNTRY_ISO   = country_iso_code            # Picked Country
RAW_INDICATORS = constants.INDICATORS       # {"INFLATION": "FP.CPI.TOTL.ZG", ...}

payload = data_retrieval.prepare_llm_payload_pretty(
    country_iso = COUNTRY_ISO,
    indicators  = RAW_INDICATORS,
    since       = 2015,
    lookback    = 10,
    deltas      = (1, 5)
)

# Turn the key → pretty-label mapping into the bullet points the prompt expects
prompt_points = ", ".join(payload["indicators"].keys())
# e.g. "Inflation (% y/y), Unemployment (% labour force), …"

# ---------------------------------------------------------------------------
# 2. Grab recent economic / political headlines -----------------------------
# ---------------------------------------------------------------------------

headlines = [
    item["title"]
    for item in fetch_links.gnews_rss(
        country  = selected_country,
        n        = 10,
        days     = 365,
        lang     = "en",
        region   = "US"
    )
]

# ---------------------------------------------------------------------------
# 3. Ask the LLM for a country risk score -----------------------------------
# ---------------------------------------------------------------------------

result = langchain_llm.country_llm_score(
    country       = payload["country"],
    headlines     = headlines,
    prompt_points = prompt_points
)

# ---------------------------------------------------------------------------
# 4. Combine & persist, ship to front-end, etc. -----------------------------
# ---------------------------------------------------------------------------

report = {
    **payload,                 # macro data
    "llm_output": result       # {"score": 0.72, "bullet_summary": "..."}
}

print(json.dumps(report, indent=2, ensure_ascii=False))

/home/linux/Projects/AI-Country-Risk-Dashboard/backend/ai/langchain_llm.py:70: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


{
  "country": "US",
  "latest_year": 2024,
  "indicators": {
    "Inflation (% y/y)": {
      "latest": 2.95,
      "Δ1y": -0.283,
      "Δ5y": 0.628,
      "series": {
        "2015": 0.12,
        "2016": 1.26,
        "2017": 2.13,
        "2018": 2.44,
        "2019": 1.81,
        "2020": 1.23,
        "2021": 4.7,
        "2022": 8.0,
        "2023": 4.12,
        "2024": 2.95
      }
    },
    "Unemployment (% labour force)": {
      "latest": 4.11,
      "Δ1y": 0.129,
      "Δ5y": 0.119,
      "series": {
        "2015": 5.28,
        "2016": 4.87,
        "2017": 4.36,
        "2018": 3.9,
        "2019": 3.67,
        "2020": 8.06,
        "2021": 5.35,
        "2022": 3.65,
        "2023": 3.64,
        "2024": 4.11
      }
    },
    "FDI inflow (% GDP)": {
      "latest": 1.33,
      "Δ1y": 0.057,
      "Δ5y": -0.094,
      "series": {
        "2015": 2.8,
        "2016": 2.52,
        "2017": 1.94,
        "2018": 1.04,
        "2019": 1.47,
        "2020": 0.64,
      

In [4]:
from backend import data_push

data_push.upsert_snapshot(report)